# EfficientNet

### Import required Libraries

In [6]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import numpy as np
import splitfolders
from tensorflow.keras.applications import EfficientNetB7

### Set Hyperparameters and Paths

In [2]:
SEED = 42
IMG_HEIGHT, IMG_WIDTH = (224, 224)
BATCH_SIZE = 64
EPOCHS = 5
output=r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split'
OUTPUT_DIR = output
train_data_dir = f"{OUTPUT_DIR}/Balanced"
valid_data_dir = f"{OUTPUT_DIR}/val"
test_data_dir = f"{OUTPUT_DIR}/test"

### Load the Data

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode="categorical")

Found 1925 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
Found 106 images belonging to 5 classes.


### Load the Model

In [7]:
base_model = EfficientNetB7(include_top=False, weights='imagenet')

258088960/258076736 [==============================] - 74s 0us/step


### Add the Layers

In [8]:
x = base_model.output
# print(x.shape)
x = GlobalAveragePooling2D()(x)
# print(x.shape)
x = Dense(1024, activation="softmax")(x)
# print(x.shape)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)
# print(predictions.shape)
model = Model(inputs=base_model.input, outputs=predictions)

### Make the Trainable Layers False

In [9]:
for layer in base_model.layers:
    layer.trainable = False

### Compile the model by selecting Appropriate Optimizer

In [10]:
model.compile(
    optimizer="pgd",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

### Train the model

In [11]:
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=3)

Epoch 1/3
31/31 [==============================] - 728s 23s/step - loss: 0.4374 - accuracy: 0.8410 - val_loss: 0.2441 - val_accuracy: 0.9400
Epoch 2/3
31/31 [==============================] - 568s 18s/step - loss: 0.1259 - accuracy: 0.9616 - val_loss: 0.3041 - val_accuracy: 0.9100
Epoch 3/3
31/31 [==============================] - 586s 19s/step - loss: 0.0583 - accuracy: 0.9813 - val_loss: 0.2865 - val_accuracy: 0.9300


### Evaluate the Test Accuracy

In [12]:
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(test_acc)

106/106 - 45s - loss: 0.2355 - accuracy: 0.9057 - 45s/epoch - 423ms/step
0.9056603908538818


### Save the EfficientNet model

In [13]:
path=r"C:\Users\GREESHMANTH\Desktop\Major Project\Models\EfficientNet.h5"
model.save(path)